In [1]:
import pandas as pd
import numpy as np

In [2]:
from src.event_sourcing_functions import removed, created, modified, incorrect_key_handler, track_changes, standardize_cols
from pathlib import Path

In [3]:
dirname = Path.cwd() / 'data'
file_paths =list(dirname.glob('*csv'))

In [4]:
snapshots = {}
dates = []
for path in file_paths:
    date = pd.read_csv(path, nrows=1, usecols=['snapshot_date']).loc[0, 'snapshot_date']
    dates.append(date)
    snapshots[date] = pd.read_csv(path)

In [5]:
for s in snapshots:
    display(snapshots[s])

,id,snapshot_date,type,colour,size,location
0,1,2024-03-31,jacket,blue,4,Vancouver
1,2,2024-03-31,shorts,gray,0,Victoria
2,3,2024-03-31,jeans,green,12,Victoria
3,4,2024-03-31,long-sleeve shirt,plaid,10,Vancouver
4,111,2024-03-31,blouse,pink,6,NaN


,id,snapshot_date,type,colour,size,featured
0,1,2024-04-30,jacket,blue,4,N
1,2,2024-04-30,shorts,gray,0,Y
2,3,2024-04-30,jeans,green,12,N
3,4,2024-04-30,long-sleeve shirt,plaid,10,Y
4,111,2024-04-30,blouse,pink,6,N


,id,snapshot_date,type,colour,size,featured
0,1.0,2024-05-31,jacket,blue,4.0,Y
1,2.0,2024-05-31,shorts,gray,0.0,Y
2,3.0,2024-05-31,jeans,green,12.0,N
3,4.0,2024-05-31,long-sleeve shirt,plaid,10.0,Y
4,111.0,2024-05-31,blouse,pink,6.0,N
5,99.0,2024-05-31,skirt,striped,NaN,NaN
6,NaN,2024-05-31,scarf,beige,NaN,NaN


,id,snapshot_date,type,colour,size,featured
0,2.0,2024-06-30,shorts,sage,0.0,Y
1,3.0,2024-06-30,jeans,green,12.0,N
2,111.0,2024-06-30,blouse,pink,6.0,N
3,5.0,2024-06-30,pants,purple,2.0,Y
4,NaN,2024-06-30,socks,gray,NaN,NaN
5,NaN,2024-06-30,scarf,beige,NaN,NaN


,id,snapshot_date,type,colour,size,featured
0,2.0,2024-07-31,shorts,sage,0.0,Y
1,3.0,2024-07-31,jeans,green,12.0,N
2,111.0,2024-07-31,blouse,pink,6.0,N
3,5.0,2024-07-31,pants,purple,2.0,Y
4,NaN,2024-07-31,socks,gray,NaN,NaN
5,6.0,2024-07-31,shirt,blue,8.0,Y
6,NaN,2024-07-31,hat,orange,NaN,NaN
7,NaN,2024-07-31,scarf,beige,NaN,NaN
8,111.0,2024-07-31,sweater,striped,10.0,N
9,1.0,2024-07-31,jacket,blue,4.0,Y


,id,snapshot_date,type,colour,size,featured
0,1.0,2024-08-31,jacket,blue,4.0,Y
1,2.0,2024-08-31,shorts,sage,0.0,Y
2,3.0,2024-08-31,jeans,green,12.0,N
3,5.0,2024-08-31,pants,purple,2.0,Y


,id,snapshot_date,type,colour,size,featured
0,1.0,2024-09-30,jacket,blue,4.0,Y
1,2.0,2024-09-30,shorts,sage,0.0,Y
2,3.0,2024-09-30,jeans,green,12.0,N
3,5.0,2024-09-30,pants,purple,2.0,Y
4,NaN,2024-09-30,vest,cream,16.0,N
5,NaN,2024-09-30,coat,brown,2.0,NaN


## 2. Functions

### 2.1 Tracking functions
Event-sourcing refers to a pattern where instead of logging everything to a certain subject, we only log events. There are no repeated rows if nothing happened before events.
Three functions track each type of change: `removed`, `created` and `modified`. Change labels appear in the `change` column along with a `change_date`. 
- *Removed* if the item is removed from the dataset. For example, id `1` appeared in May but it does not appear in the June dataset. A copy of the removed row is stored in the dataset along with the date. In this example, May record for id 1 will appear in the change log with the snapshot date in May and the `change_date` in June and a `removed` entry in the `change` column.
**Types of changes**
- *Created* when the key appears for the first time in the dataset or when it appears again after being removed.
- *Modified* 
    - if something changes in the record. For example, id `1` was labelled as `N` in the `featured` field in April but it changed to `Y` in May.
    - If a column is created or removed. For instance, March snapshot contains a `location` field but it is removed later on.

### 2.2 Data wrangling functions

Event sourcing process depends on valid keys. Incorrect keys include duplicate or missing keys. They require special handling.
- During each comparison between a new and a previous dataset, duplicate keys are recorded as an output of the `incorrect_key_handler` function.
    - This function also outputs both new and old datasets without missing or duplicate keys.

Fields may change between snapshots. To accommodate that, each new field is filled with NAs in the previous/old dataset. Any fields that have been removed are filled with NAs for the new/current dataset.
- `standardize_cols` function achieves this purpose.

## 3. Tracking changes

`track_changes` function puts five functions mentioned above together (along with a minor function for reordering columns).

In [6]:
removed(snapshots[dates[2]], snapshots[dates[3]], 'id', 'snapshot_date')

,id,snapshot_date,type,colour,size,featured,change,change_date
0,1.0,2024-05-31,jacket,blue,4.0,Y,removed,2024-06-30
3,4.0,2024-05-31,long-sleeve shirt,plaid,10.0,Y,removed,2024-06-30
5,99.0,2024-05-31,skirt,striped,NaN,NaN,removed,2024-06-30
6,NaN,2024-05-31,scarf,beige,NaN,NaN,removed,2024-06-30


In [7]:
created(snapshots[dates[3]], 'id', 'snapshot_date', old = snapshots[dates[2]])

,id,snapshot_date,type,colour,size,featured,change,change_date
3,5.0,2024-06-30,pants,purple,2.0,Y,created,2024-06-30
4,NaN,2024-06-30,socks,gray,NaN,NaN,created,2024-06-30
5,NaN,2024-06-30,scarf,beige,NaN,NaN,created,2024-06-30


In [8]:
created(snapshots[dates[0]], 'id', 'snapshot_date') # Example of a dataset that is the first of its series

,id,snapshot_date,type,colour,size,location,change,change_date
0,1,2024-03-31,jacket,blue,4,Vancouver,created,2024-03-31
1,2,2024-03-31,shorts,gray,0,Victoria,created,2024-03-31
2,3,2024-03-31,jeans,green,12,Victoria,created,2024-03-31
3,4,2024-03-31,long-sleeve shirt,plaid,10,Vancouver,created,2024-03-31
4,111,2024-03-31,blouse,pink,6,NaN,created,2024-03-31


In [9]:
modified(snapshots[dates[1]], snapshots[dates[2]], 'id', 'snapshot_date') # Example

,id,snapshot_date,type,colour,size,featured,change,change_date
0,1.0,2024-05-31,jacket,blue,4.0,Y,modified,2024-05-31


Below is an example of the `incorrect_key_handler`. June snapshot contains two NA ids. July snapshot contains three 111 ids and three NA ids.

In [10]:
snapshots[dates[3]]

,id,snapshot_date,type,colour,size,featured
0,2.0,2024-06-30,shorts,sage,0.0,Y
1,3.0,2024-06-30,jeans,green,12.0,N
2,111.0,2024-06-30,blouse,pink,6.0,N
3,5.0,2024-06-30,pants,purple,2.0,Y
4,NaN,2024-06-30,socks,gray,NaN,NaN
5,NaN,2024-06-30,scarf,beige,NaN,NaN


In [11]:
snapshots[dates[4]]

,id,snapshot_date,type,colour,size,featured
0,2.0,2024-07-31,shorts,sage,0.0,Y
1,3.0,2024-07-31,jeans,green,12.0,N
2,111.0,2024-07-31,blouse,pink,6.0,N
3,5.0,2024-07-31,pants,purple,2.0,Y
4,NaN,2024-07-31,socks,gray,NaN,NaN
5,6.0,2024-07-31,shirt,blue,8.0,Y
6,NaN,2024-07-31,hat,orange,NaN,NaN
7,NaN,2024-07-31,scarf,beige,NaN,NaN
8,111.0,2024-07-31,sweater,striped,10.0,N
9,1.0,2024-07-31,jacket,blue,4.0,Y


After applying the function missing and duplicate keys are removed.

In [12]:
incorrect_key_handler(snapshots[dates[4]], 'id', old = snapshots[dates[3]])['old'] 

,id,snapshot_date,type,colour,size,featured
0,2.0,2024-06-30,shorts,sage,0.0,Y
1,3.0,2024-06-30,jeans,green,12.0,N
3,5.0,2024-06-30,pants,purple,2.0,Y


In [13]:
incorrect_key_handler(snapshots[dates[4]], 'id', old = snapshots[dates[3]])['new'] 

,id,snapshot_date,type,colour,size,featured
0,2.0,2024-07-31,shorts,sage,0.0,Y
1,3.0,2024-07-31,jeans,green,12.0,N
3,5.0,2024-07-31,pants,purple,2.0,Y
5,6.0,2024-07-31,shirt,blue,8.0,Y
9,1.0,2024-07-31,jacket,blue,4.0,Y


Incorrect keys are also output as a set.

In [14]:
incorrect_key_handler(snapshots[dates[4]], 'id', old = snapshots[dates[3]])['new_duplicate_set']

{111.0}

Below is an example of standardizing columns. March (old) dataset contains a `location` field and no `featured` column. April contains a `featured` column but no `location`.
Both columns are kept for comparison  with no 

In [15]:
standardize_cols(snapshots[dates[1]], snapshots[dates[0]])['old']

,id,snapshot_date,type,colour,size,featured,location
0,1,2024-03-31,jacket,blue,4,NaN,Vancouver
1,2,2024-03-31,shorts,gray,0,NaN,Victoria
2,3,2024-03-31,jeans,green,12,NaN,Victoria
3,4,2024-03-31,long-sleeve shirt,plaid,10,NaN,Vancouver
4,111,2024-03-31,blouse,pink,6,NaN,NaN


In [16]:
standardize_cols(snapshots[dates[1]], snapshots[dates[0]])['new']

,id,snapshot_date,type,colour,size,featured,location
0,1,2024-04-30,jacket,blue,4,N,NaN
1,2,2024-04-30,shorts,gray,0,Y,NaN
2,3,2024-04-30,jeans,green,12,N,NaN
3,4,2024-04-30,long-sleeve shirt,plaid,10,Y,NaN
4,111,2024-04-30,blouse,pink,6,N,NaN


## 3. Putting it all together
`track_changes` function will get rid of problematic edge cases:
- Missing keys
- Duplicate keys
- Added columns
- Removed columns

It will put rows associated with the wrong keys into a separate dataset that does not track changes.
Rows with unique keys will be passed into the `created`, `removed` and `modified` functions. All columns will be stored in the final dataset.
The output of these functions will be merged together.

### 3.1 Tracking one or two datasets.

In this example, the snapshot if the first in the series. All rows are labelled as created.

In [17]:
track_changes(snapshots[dates[0]], 'id', 'snapshot_date')['changes_df']

,id,colour,location,size,snapshot_date,type,change_date,change
0,1,blue,Vancouver,4,2024-03-31,jacket,2024-03-31,created
1,2,gray,Victoria,0,2024-03-31,shorts,2024-03-31,created
2,3,green,Victoria,12,2024-03-31,jeans,2024-03-31,created
3,4,plaid,Vancouver,10,2024-03-31,long-sleeve shirt,2024-03-31,created
4,111,pink,NaN,6,2024-03-31,blouse,2024-03-31,created


In the example below a variety of changes are tracked between May (old) and June (new) snapshots:

In [18]:
track_changes(snapshots[dates[3]], 'id', 'snapshot_date', old=snapshots[dates[2]])['changes_df']

,id,colour,featured,size,snapshot_date,type,change_date,change
0,1.0,blue,Y,4.0,2024-05-31,jacket,2024-06-30,removed
1,2.0,sage,Y,0.0,2024-06-30,shorts,2024-06-30,modified
2,4.0,plaid,Y,10.0,2024-05-31,long-sleeve shirt,2024-06-30,removed
3,5.0,purple,Y,2.0,2024-06-30,pants,2024-06-30,created
4,99.0,striped,NaN,NaN,2024-05-31,skirt,2024-06-30,removed
5,NaN,gray,NaN,NaN,2024-06-30,socks,2024-06-30,untracked_missing_key
6,NaN,beige,NaN,NaN,2024-06-30,scarf,2024-06-30,untracked_missing_key


### 3.2. Tracking all of the toy datasets
Going through all snapshots:

In [19]:
tracker_output = {}
tracker_output[0] = track_changes(snapshots[dates[0]], 'id', 'snapshot_date')
for i in range(0,  len(dates)-1):
    if (i == len(dates)-1):
        break
    else:
        tracker_output[i+1] = track_changes(
            snapshots[dates[i+1]], 
            'id', 'snapshot_date',
            tracker_output[i]['new_duplicate_set'],
            old = snapshots[dates[i]])    

In [20]:
complete_df = pd.DataFrame()
for i in range(0, len(tracker_output)):
    complete_df = pd.concat(
        [
            complete_df,
            tracker_output[i]['changes_df']
        ]
    )

In [21]:
display(complete_df.reset_index(drop=True))

,id,colour,location,size,snapshot_date,type,change_date,change,featured
0,1.0,blue,Vancouver,4.0,2024-03-31,jacket,2024-03-31,created,NaN
1,2.0,gray,Victoria,0.0,2024-03-31,shorts,2024-03-31,created,NaN
2,3.0,green,Victoria,12.0,2024-03-31,jeans,2024-03-31,created,NaN
3,4.0,plaid,Vancouver,10.0,2024-03-31,long-sleeve shirt,2024-03-31,created,NaN
4,111.0,pink,NaN,6.0,2024-03-31,blouse,2024-03-31,created,NaN
5,1.0,blue,NaN,4.0,2024-04-30,jacket,2024-04-30,modified,N
6,2.0,gray,NaN,0.0,2024-04-30,shorts,2024-04-30,modified,Y
7,3.0,green,NaN,12.0,2024-04-30,jeans,2024-04-30,modified,N
8,4.0,plaid,NaN,10.0,2024-04-30,long-sleeve shirt,2024-04-30,modified,Y
9,111.0,pink,NaN,6.0,2024-04-30,blouse,2024-04-30,modified,N


In [22]:
tracker_output[6]['new_duplicate_set']

{111.0}